In [1]:
%%bash
$HADOOP_HOME/sbin/start-dfs.sh
$HADOOP_HOME/sbin/start-yarn.sh
$HADOOP_HOME/bin/mapred --daemon start historyserver

Starting namenodes on [localhost]
Starting datanodes
Starting secondary namenodes [p-ec4ea129-ae6a-44e4-85a6-40ee24c22258]
Starting resourcemanager
Starting nodemanagers


In [2]:
! $HADOOP_HOME/bin/hdfs dfs -put salaries.csv

put: `salaries.csv': File exists


In [3]:
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(host='localhost', port=9000)
hdfs.ls('.')

['/user/jovyan/.sparkStaging',
 '/user/jovyan/output',
 '/user/jovyan/output2',
 '/user/jovyan/salaries.csv']

In [6]:
%%file salaries_spark.py

from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import expr
from pyspark.sql.types import *
from pyspark.sql.functions import *
from functools import reduce

# SAMPLE LINE:
#"Aaron,Keontae E",AIDE BLUE CHIP,W02200,Youth Summer  ,06/10/2013,$11310.00,$873.63

spark = SparkSession.builder.appName("Top Salaries").getOrCreate()

mySchema = StructType([
    StructField("a", StringType(), False),
    StructField("b", StringType(), False),
    StructField("c", StringType(), False),
    StructField("d", StringType(), False),
    StructField("e", StringType(), False),
    StructField("salary", StringType(), False),
    StructField("f", StringType(), False)
    ])

df = spark.read.format("csv").option("header","true").schema(mySchema).load("salaries.csv")
excluColumns = "a,b,c,d,e,f".split(",")
df = reduce(DataFrame.drop, excluColumns, df).withColumn('d_salary', regexp_replace('salary', '[$,]', '').cast('double'))
excluColumns= ["salary"]
df2 = reduce(DataFrame.drop, excluColumns, df)
df2.show()
result = df2.sort(desc("d_salary")).limit(10)
result.write.format("csv").option("path", "output3").save()



Overwriting salaries_spark.py


In [7]:
! $SPARK_HOME/bin/spark-submit --master yarn salaries_spark.py

2020-11-30 10:37:59,644 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-11-30 10:38:00,624 INFO spark.SparkContext: Running Spark version 3.0.1
2020-11-30 10:38:00,709 INFO resource.ResourceUtils: ==============================================================
2020-11-30 10:38:00,713 INFO resource.ResourceUtils: Resources for spark.driver:

2020-11-30 10:38:00,713 INFO resource.ResourceUtils: ==============================================================
2020-11-30 10:38:00,715 INFO spark.SparkContext: Submitted application: Top Salaries
2020-11-30 10:38:00,819 INFO spark.SecurityManager: Changing view acls to: jovyan
2020-11-30 10:38:00,820 INFO spark.SecurityManager: Changing modify acls to: jovyan
2020-11-30 10:38:00,820 INFO spark.SecurityManager: Changing view acls groups to: 
2020-11-30 10:38:00,820 INFO spark.SecurityManager: Changing modify acls groups to: 
2020-11-30 10:38:00,820 INFO spark.S

In [8]:
! $HADOOP_HOME/bin/hdfs dfs -get output3 .

In [ ]:
%%bash
$HADOOP_HOME/bin/mapred --daemon stop historyserver
$HADOOP_HOME/sbin/stop-yarn.sh
$HADOOP_HOME/sbin/stop-dfs.sh